In [1]:
%reload_ext autoreload
%autoreload 2

from IPython.core.display import Markdown
from tqdm.auto import tqdm
import config
import docs
import pandas as pd
import requests
import acs

pd.set_option('display.max_colwidth', None)

INDICATOR = 'telework_rate'
CONFIG = config.get_config(INDICATOR, '../config.toml')
CENSUS_API_KEY = CONFIG['acs_api_key']

display(Markdown('## Raw data path'))
raw_dir_path = str(CONFIG['raw_dir']).replace('\\', '/')
display(Markdown(f"[{raw_dir_path}]({raw_dir_path})"))


c:\Users\tan\src\regional-pm-2023\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Raw data path

[C:/Users/tan/src/regional-pm-2023/data/raw/qol/telework/telework_rate](C:/Users/tan/src/regional-pm-2023/data/raw/qol/telework/telework_rate)

# Transportation Planning: Commute

## Commute Mode

In [2]:
docs.describe_indicator(
    indicator=INDICATOR,
    indicators_xlsx_path=CONFIG['indicators_xlsx_path'],
)

Percentage of residents that telework.

nan

In [3]:
docs.list_schema(
    indicator=INDICATOR,
    indicators_xlsx_path=CONFIG['indicators_xlsx_path'],
)

,name,description,type
column,,,
year,Year,Year of record.,datetime
telework_rate,Telework Rate,Percentage of residents that telework,float


In [4]:
docs.list_sources(
    indicator=INDICATOR,
    indicators_xlsx_path=CONFIG['indicators_xlsx_path'],
)

,name,organization,active,notes
source,,,,
telework,American Community Survey API (B08301),US Census Bureau,True,"Ultimately, this is a single column from the Commute Mode indicator"


In [5]:
steps = docs.list_update_steps(
    indicator=INDICATOR,
    indicators_xlsx_path=CONFIG['indicators_xlsx_path'],
)
steps

,step
0,Download raw ACS data.
1,Transform raw ACS data.


In [6]:
docs.list_remarks(
    indicator=INDICATOR,
    indicators_xlsx_path=CONFIG['indicators_xlsx_path'],
)

,author,note
0,NaN,NaN


### Step 0: Extract from commute mode data

In [8]:
display(steps.loc[0])

step    Download raw ACS data.
Name: 0, dtype: object

In [15]:
telework_rate = (
    pd.read_csv(CONFIG['raw_dir']/'commute_mode_odp.csv')
    .set_index(['year', 'geography', 'mode'])
    .xs('Work at Place of Residence', level='mode')
    .rename(columns={'mode_share': 'telework_rate'})
)
telework_rate

telework_rate
year       geography                      
2005-01-01 San Diego County       0.044625
2006-01-01 San Diego County       0.065321
2007-01-01 San Diego County       0.062024
2008-01-01 San Diego County       0.058409
2009-01-01 San Diego County       0.066102
2010-01-01 San Diego County       0.058956
           United States          0.043261
2011-01-01 San Diego County       0.063213
           United States          0.043351
2012-01-01 San Diego County       0.065868
           United States          0.043616
2013-01-01 San Diego County       0.063845
           United States          0.043571
2014-01-01 San Diego County       0.075133
           United States          0.044852
2015-01-01 San Diego County       0.068907
           United States          0.046135
2016-01-01 San Diego County       0.071487
           United States          0.050485
2017-01-01 San Diego County       0.069437
           United States          0.052318
2018-01-01 San Diego County       0.066001
           United States          0.053372
2019-01-01 San Diego County       0.074977
           United States          0.057160
2021-01-01 San Diego County       0.221201
           United States          0.178649

### Save Data

In [14]:
display(Markdown('#### Clean data path'))
clean_dir_path = str(CONFIG['clean_dir']).replace('\\', '/')
display(Markdown(f"[{clean_dir_path}]({clean_dir_path})"))

#### Clean data path

[C:/Users/tan/src/regional-pm-2023/data/clean/transportation/commute/commute_mode](C:/Users/tan/src/regional-pm-2023/data/clean/transportation/commute/commute_mode)

In [15]:
telework_rate.to_csv(
    CONFIG['clean_dir']
    / f'{INDICATOR}_odp.csv'
)